# Chinese data reading

In [2]:
import pyaerocom as pya
import os
import csv
from glob import glob
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np

In [3]:
path = '/lustre/storeA/project/aerocom/aerocom1/AEROCOM_OBSDATA/CHINA_SON2020_MP_NRT'

#### using pandas

In [15]:
#initialize empty dataframe
data = pd.DataFrame()

#loop over files
files = glob(os.path.join(path, '*.csv'))
for i in tqdm(range(len(files))):
#for i in tqdm(range(2)):
    fn = os.path.join(path,files[i])
    data = data.append(pd.read_csv(fn,sep=','))
    
data['dateTime'] = pd.to_datetime(data['dateTime'].values)
data.head()

100%|██████████| 4/4 [07:34<00:00, 113.73s/it]


,station_ID,dateTime,species,value
0,3728A,2020-09-30 23:00:00,co,800.0
1,3728A,2020-09-30 23:00:00,so2,4.0
2,3728A,2020-09-30 23:00:00,no2,19.0
3,3728A,2020-09-30 23:00:00,o3,64.0
4,3728A,2020-09-30 23:00:00,pm10,85.0


#### check what the data looks like

In [13]:
#plot some time series
import matplotlib.pyplot as plt

var = 'o3'
station_id = '3728A'
sub_data = data.loc[(data['species'] == var) & (data['station_ID'] == station_id)]

fig=plt.figure(figsize=(18,6), dpi= 100, facecolor='w', edgecolor='k')
plt.plot(sub_data['dateTime'],sub_data['value'],'o')
plt.ylabel(sub_data['species'].values[0])
plt.title(sub_data['station_ID'].values[0])

KeyError: 'station_ID'

#### configuration file

In [6]:
fn = os.path.join(path,'station_info.xlsx')
cfg = pd.read_excel(fn,engine='openpyxl')
cfg.head()

,stationId,stationName,cityId,latitude,longitude,cityName
0,1001A,万寿西宫,1,39.8673,116.366,北京
1,1002A,定陵,1,40.2865,116.170,北京
2,1003A,东四,1,39.9522,116.434,北京
3,1004A,天坛,1,39.8745,116.434,北京
4,1005A,农展馆,1,39.9716,116.473,北京


#### Fill data with cfg

In [7]:
#in order to merge database, set common index with station id
data.set_index('station_ID', inplace=True)
cfg.rename(columns={'stationId': 'station_ID'},inplace=True)
cfg.set_index('station_ID',inplace=True)

In [ ]:
data_cfg = data.join(cfg, how="inner")
data_cfg.reset_index(inplace=True)
print(len(data),len(data_cfg))
data_cfg.head()

#### explore a little bit more the data

In [9]:
var = 'o3'
sub_data = data_cfg.loc[data_cfg['species']==var]
sub_data.rename(columns={'dateTime': 'datetime'},inplace=True)
sub_data.drop(columns=['species','cityId','cityName','stationName'], inplace=True)

/home/augustinm/anaconda3/envs/pya/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/home/augustinm/anaconda3/envs/pya/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
from keplergl import KeplerGl
kepler_map = KeplerGl(height = 800, data={var: sub_data})
kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'o3':          station_ID            datetime  value  latitude  longitude
3             1001A 2…